# Final CHO Model
This notebook is to asses the validity of our reconstruction and how complete it is.

In [1]:
# Import libraries

import gspread
import pandas as pd
import numpy as np

from cobra import Model, Reaction, Metabolite
from cobra.io import validate_sbml_model

from tqdm.notebook import tqdm

/Users/pablodigiusto/miniforge3/envs/mncho/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


### 1. Retrieve information from the Google Sheet datasets reactions and metabolites

In [2]:
# give service account details to gspread
sa = gspread.service_account(filename='credentials.json')

# sa is a gspread client, which can be used for connecting to the sheets
# by using the open method and the sheet name.
cho_recon = sa.open('CHO Network Reconstruction')

# we also need to specify the page name before getting the data. In this case we use the Rxns and Metabolites sheet.
rxns_sheet = cho_recon.worksheet('Rxns')
rxns_attributes_sheet = cho_recon.worksheet('Attributes')
metabolites_sheet = cho_recon.worksheet('Metabolites')

# We can extract the data using the get_all_records method and create pd DataFrames

# Reactions IDs, names, formulas, GPRs
rxns = pd.DataFrame(rxns_sheet.get_all_records())

# Reactions bounds
rxns_attributes = pd.DataFrame(rxns_attributes_sheet.get_all_records())

# Metabolites names, formulas and compartment
metabolites = pd.DataFrame(metabolites_sheet.get_all_records())

### 2. Build a model and feed it the information from the df generated

In [3]:
# Create a model and add reactions

model = Model("iCHO")
lr = []

for _, row in rxns.iterrows():
    r = Reaction(row['Reaction'])
    lr.append(r)
    
model.add_reactions(lr)
model

Name,iCHO
Memory address,13ee4f730
Number of metabolites,0
Number of reactions,8189
Number of genes,0
Number of groups,0
Objective expression,0
Compartments,


In [4]:
# Add information to each one of the reactions

for i,r in enumerate(tqdm(model.reactions)):
    r.build_reaction_from_string(rxns['Reaction Formula'][i])
    r.name = rxns['Reaction Name'][i]
    r.subsystem = rxns['Subsystem'][i]
    r.gene_reaction_rule = str(rxns['GPR_final'][i])
    r.lower_bound = rxns_attributes['Lower bound'][i]
    r.upper_bound = rxns_attributes['Upper bound'][i]

  0%|          | 0/8189 [00:00<?, ?it/s]

unknown metabolite '10fthf5glu_c' created
unknown metabolite '10fthf5glu_l' created
unknown metabolite '10fthf5glu_m' created
unknown metabolite '10fthf6glu_c' created
unknown metabolite '10fthf6glu_l' created
unknown metabolite '10fthf6glu_m' created
unknown metabolite '10fthf7glu_c' created
unknown metabolite '10fthf7glu_l' created
unknown metabolite '10fthf7glu_m' created
unknown metabolite '10fthf_c' created
unknown metabolite '10fthf_l' created
unknown metabolite '10fthf_m' created
unknown metabolite '11docrtsl_c' created
unknown metabolite '11docrtsl_m' created
unknown metabolite '11docrtsl_r' created
unknown metabolite '11docrtstrn_c' created
unknown metabolite '11docrtstrn_m' created
unknown metabolite '11docrtstrn_r' created
unknown metabolite '12HPET_c' created
unknown metabolite 'atp_c' created
unknown metabolite 'h2o_c' created
unknown metabolite '12HPET_e' created
unknown metabolite 'adp_c' created
unknown metabolite 'h_c' created
unknown metabolite 'pi_c' created
unknown 

unknown metabolite '3mox4hpac_c' created
unknown metabolite 'mercppyr_c' created
unknown metabolite 'so3_c' created
unknown metabolite 'otsul_c' created
unknown metabolite '3mlda_c' created
unknown metabolite '3mlda_e' created
unknown metabolite '3mob_c' created
unknown metabolite '3mob_m' created
unknown metabolite '3mob_e' created
unknown metabolite '3mop_c' created
unknown metabolite '3mop_e' created
unknown metabolite '3mox4hoxm_c' created
unknown metabolite '3mox4hoxm_e' created
unknown metabolite '3m4hpga_c' created
unknown metabolite '3moxtyr_c' created
unknown metabolite '3amp_l' created
unknown metabolite 'h2o_l' created
unknown metabolite 'adn_l' created
unknown metabolite 'pi_l' created
unknown metabolite '3odecACP_c' created
unknown metabolite '3odecACP_m' created
unknown metabolite '3oddecACP_c' created
unknown metabolite '3oddecACP_m' created
unknown metabolite '3omrsACP_c' created
unknown metabolite '3omrsACP_m' created
unknown metabolite '3opalmACP_c' created
unknown me

unknown metabolite 'crvncoa_m' created
unknown metabolite 'nrvnccoa_x' created
unknown metabolite 'tettet6coa_m' created
unknown metabolite 'tettet6coa_x' created
unknown metabolite 'tetpent6coa_m' created
unknown metabolite 'tetpent6coa_x' created
unknown metabolite 'tethex3coa_x' created
unknown metabolite 'hexccoa_x' created
unknown metabolite 'ivcoa_m' created
unknown metabolite '3mb2coa_m' created
unknown metabolite '2mp2coa_m' created
unknown metabolite 'Sdmhptcoa_m' created
unknown metabolite 'Sdmnoncoa_m' created
unknown metabolite 'mptcoa_m' created
unknown metabolite 'Spristcoa_x' created
unknown metabolite 'Stmhndccoa_x' created
unknown metabolite 'Stmtridccoa_x' created
unknown metabolite 'dcacoa_c' created
unknown metabolite 'dca_c' created
unknown metabolite 'dca_m' created
unknown metabolite 'dca_x' created
unknown metabolite 'ddcacoa_c' created
unknown metabolite 'ddca_c' created
unknown metabolite 'ddca_m' created
unknown metabolite 'ddca_x' created
unknown metabolite 

unknown metabolite 'R2coa_cho_c' created
unknown metabolite 'tmndnccoa_c' created
unknown metabolite 'clpndcoa_c' created
unknown metabolite 'dcsptn1coa_c' created
unknown metabolite 'c226coa_c' created
unknown metabolite 'stcoa_c' created
unknown metabolite 'R3coa_cho_c' created
unknown metabolite 'odecoa_c' created
unknown metabolite 'vacccoa_c' created
unknown metabolite 'R4coa_cho_c' created
unknown metabolite 'nrvnccoa_c' created
unknown metabolite 'lgnccoa_c' created
unknown metabolite 'R5coa_cho_c' created
unknown metabolite 'hexccoa_c' created
unknown metabolite 'eicostetcoa_c' created
unknown metabolite 'tetpent6coa_c' created
unknown metabolite 'tetpent3coa_c' created
unknown metabolite 'tettet6coa_c' created
unknown metabolite 'tethex3coa_c' created
unknown metabolite 'hdd2coa_c' created
unknown metabolite 'R6coa_cho_c' created
unknown metabolite 'Rtotalcoa_m' created
unknown metabolite 'Rtotal3coa_m' created
unknown metabolite 'crm_cho_l' created
unknown metabolite 'Rtotal_

...

unknown metabolite 'CE5946_c' created
unknown metabolite 'CE5945_r' created
unknown metabolite 'CE5946_r' created
unknown metabolite 'CE5947_c' created
unknown metabolite 'CE7074_c' created
unknown metabolite 'CE7072_c' created
unknown metabolite 'CE7074_r' created
unknown metabolite 'CE7072_r' created
unknown metabolite 'CE5966_m' created
unknown metabolite 'CE5967_m' created
unknown metabolite 'CE5966_x' created
unknown metabolite 'CE5967_x' created
unknown metabolite 'CE5967_c' created
unknown metabolite 'CE5968_c' created
unknown metabolite 'CE5968_m' created
unknown metabolite 'CE5968_x' created
unknown metabolite 'CE5969_m' created
unknown metabolite 'CE5970_m' created
unknown metabolite 'CE5969_x' created
unknown metabolite 'CE5970_x' created
unknown metabolite 'CE5969_c' created
unknown metabolite 'CE5969_r' created
unknown metabolite 'CE5971_m' created
unknown metabolite 'CE5971_x' created
unknown metabolite 'CE5971_c' created
unknown metabolite 'CE5966_c' created
unknown meta

unknown metabolite 'CN0016_r' created
unknown metabolite 'CN0017_r' created
unknown metabolite 'CN0018_c' created
unknown metabolite 'CN0018_r' created
unknown metabolite 'CN0017_x' created
unknown metabolite 'CN0018_x' created
unknown metabolite 'CN0019_c' created
unknown metabolite 'CN0019_r' created
unknown metabolite 'rna_c' created
unknown metabolite 'rna_prod_c' created
unknown metabolite 'HC02199_c' created
unknown metabolite 'HC02200_c' created
unknown metabolite 'HC02201_c' created
unknown metabolite 'xu5p_D_c' created
unknown metabolite 'Rtotal2crn_c' created
unknown metabolite 'Rtotal2crn_m' created
unknown metabolite 'Rtotal3crn_c' created
unknown metabolite 'Rtotal3crn_m' created
unknown metabolite 'Rtotalcrn_c' created
unknown metabolite 'Rtotalcrn_m' created
unknown metabolite 'sl2n2m2masn_g' created
unknown metabolite 's3lnl3fn3m2masn_g' created
unknown metabolite 's4lndl4fn4m2masn_g' created
unknown metabolite 's4lndl4n4m2masn_g' created
unknown metabolite 's4lncl4fn4m

unknown metabolite 'dhcholestanate_r' created
unknown metabolite 'dhcholestancoa_r' created
unknown metabolite 'thcholstoic_r' created
unknown metabolite 'cholcoar_r' created
unknown metabolite 'wharachd_c' created
unknown metabolite 'whtststerone_c' created
unknown metabolite 'xan_x' created
unknown metabolite 'xol7ah2_r' created
unknown metabolite 'xoldiolone_m' created
unknown metabolite 'xoltri24_c' created
unknown metabolite 'xoltri25_c' created
unknown metabolite 'xser_r' created
unknown metabolite 'xyl_D_c' created
unknown metabolite 'xylnact_D_c' created
unknown metabolite 'xylt_c' created
unknown metabolite 'zn2_c' created
unknown metabolite 'HC02119_c' created
unknown metabolite 'HC00822_l' created
unknown metabolite 'dpcoa_m' created
unknown metabolite 'HC01672_c' created
unknown metabolite 'HC01434_c' created
unknown metabolite 'HC01434_m' created
unknown metabolite 'HC01434_x' created
unknown metabolite 'HC00591_c' created
unknown metabolite 'HC00591_m' created
unknown met

Malformed gene_reaction_rule '__cobra_escape__100764088 or __cobra_escape__100763802 or __cobra_escape__100763802 or h728441 or h102724197 or __cobra_escape__100752303 or __cobra_escape__100758668) or ((__cobra_escape__100764088 or __cobra_escape__100763802 or __cobra_escape__100763802 or h728441 or h102724197 or __cobra_escape__100752303 or __cobra_escape__100758668) and __cobra_escape__100763802' for 100764088 or 100763802 or 100763802 or h728441 or h102724197 or 100752303 or 100758668) or ((100764088 or 100763802 or 100763802 or h728441 or h102724197 or 100752303 or 100758668) and 100763802
Traceback (most recent call last):
  File "/Users/pablodigiusto/miniforge3/envs/mncho/lib/python3.10/site-packages/cobra/core/gene.py", line 350, in from_string
    tree = ast_parse(escaped_str, "<string>", "eval")
  File "/Users/pablodigiusto/miniforge3/envs/mncho/lib/python3.10/ast.py", line 50, in parse
    return compile(source, filename, mode, flags,
  File "<string>", line 1
    __cobra_esc

unknown metabolite 'c8crn_x' created
unknown metabolite 'HC00900_c' created
unknown metabolite 'C02470_m' created
unknown metabolite 'C02528_r' created
unknown metabolite 'tetdece1coa_m' created
unknown metabolite 'HC01231_m' created
unknown metabolite 'HC01710_c' created
unknown metabolite '3hdcoa_x' created
unknown metabolite 'xol7ah3_c' created
unknown metabolite 'HC01459_x' created
unknown metabolite '34dhmald_m' created
unknown metabolite '34dhoxmand_m' created
unknown metabolite '3mox4hpac_m' created
unknown metabolite 'homoval_m' created
unknown metabolite '3m4hpga_m' created
unknown metabolite '3mox4hoxm_m' created
unknown metabolite 'HC01652_c' created
unknown metabolite 'HC02228_c' created
unknown metabolite 'HC01842_c' created
unknown metabolite 'HC01797_c' created
unknown metabolite 'HC00004_c' created
unknown metabolite 'HC00319_c' created
unknown metabolite 'HC00319_m' created
unknown metabolite 'arachd_l' created
unknown metabolite 'hdca_l' created
unknown metabolite 'na

In [5]:
# Add information for each metabolite
metabolites_dict = metabolites.set_index('BiGG ID').to_dict('dict')
metabolites_dict['Name']

model_met_list = [] #Generate a list of all the metabolites in the model
gs_met_list = list(metabolites['BiGG ID']) #Generate a list of all the metabolites in the Google Sheet file

for met in model.metabolites:
    met.name = metabolites_dict['Name'][f'{met}']
    met.formula = metabolites_dict['Formula'][f'{met}']
    met.compartment = metabolites_dict['Compartment'][f'{met}'].split(' - ')[0]
    model_met_list.append(met.id)

model

Name,iCHO
Memory address,13ee4f730
Number of metabolites,5431
Number of reactions,8189
Number of genes,1403
Number of groups,0
Objective expression,0
Compartments,"c, l, m, r, e, x, n, g, , im"


### 3. Check the model

In [6]:
# Check that the 

print(len(gs_met_list))
print(len(model_met_list))

for m in gs_met_list:
    if m not in model_met_list:
        print(f'Metabolite {m} in the Google Sheet file not in the Model')
              
for m in model_met_list:
    if m not in gs_met_list:
        print(f'Metabolite {m} in the Model not in the Google Sheet file')

5431
5431


In [7]:
# Check Mass Balance
i = 0
for rxn in model.reactions:
    if len(rxn.check_mass_balance()) != 0:
        print(rxn.id)
        print(rxn.reaction)
        print(rxn.check_mass_balance())
        i+=1
        print(i)
        print('.......................................')

15KPROSTGF2c
nad_c + prostgf2_c <=> 15kprostgf2_c + nadh_c
{'H': -1}
1
.......................................
2AMACSULT
2amac_c + nadph_c + paps_c --> Lcyst_c + nadp_c + pap_c
{'O': 3, 'S': 1}
2
.......................................
3HAD141n7m
c142_2Z_7Zcoa_m + h2o_m <=> 3hoc141_7Zcoa_m
{'H': -4}
3
.......................................
3HAD260p
h2o_x + hex2coa_x <=> 3hhexcoa_x
{'H': 4}
4
.......................................
3HDH141n7m
3hoc141_7Zcoa_m + nad_m <=> 3oc141_7Zcoa_m + h_m + nadh_m
{'H': 4}
5
.......................................
4NPHSULT
4nph_c + paps_c --> 4nphsf_c + h_c + pap_c
{'O': 3, 'S': 1}
6
.......................................
5ADTSTSTERONESULT
5adtststerone_c + paps_c --> 5adtststerones_c + h_c + pap_c
{'O': 3, 'S': 1}
7
.......................................
A4GALTc
galgluside_cho_c + udpgal_c --> h_c + thcrm_cho_c + udp_c
{'F': 1, 'U': 1, 'L': 2}
8
.......................................
ACACT226n3m
3oc226_4Z_7Z_10Z_13Z_16Z_19Zcoa_m + coa_m --> accoa

In [8]:
# Initiliase model
for rxn_exchange in model.exchanges:
    rxn_exchange.bounds = (-1000, 1000)
bio_id = model.reactions.index('biomass_producing')
atp_id = model.reactions.index('DM_atp_c')

model.reactions[bio_id].bounds = (0, 100)
model.objective = 'biomass_producing'
# model.objective = 'DM_atp_c'
model.optimize()

,fluxes,reduced_costs
10FTHF5GLUtl,0.0,0.000000e+00
10FTHF5GLUtm,0.0,0.000000e+00
10FTHF6GLUtl,0.0,-9.860761e-32
10FTHF6GLUtm,0.0,0.000000e+00
10FTHF7GLUtl,0.0,-9.860761e-32
...,...,...
r2537,0.0,-0.000000e+00
r2538,0.0,-0.000000e+00
r2539,0.0,-0.000000e+00
ALLTTtm,0.0,0.000000e+00


In [ ]:
with open('Biomass_Metabolites_Reactions.txt', 'w') as f:
    bio_mets = model.reactions.get_by_id('biomass_producing').metabolites
    for bio_met in bio_mets:
        f.write("Metabolite: " + bio_met.name + "\n")
        f.write("Reactions: " + ", ".join([r.name for r in bio_met.reactions]) + "\n")
        f.write("Formula: " + ", ".join([r.build_reaction_string() for r in bio_met.reactions]) + "\n\n")

# Detect all the excahnge reactions that have no uptake
from cobra.flux_analysis import flux_variability_analysis
FVA_Result = flux_variability_analysis(model, model.exchanges)

#FVA_Result = a[a.minimum == 0]
with open('FVA_Excange.txt', 'w') as f:
    f.write("reaction\tminimum\tmaximum\n")
    for index, row in FVA_Result.iterrows():
        f.write(f"{index}\t{row['minimum']}\t{row['maximum']}\n")

In [9]:
for reaction in model.reactions:
    with model as model:
        reaction.knock_out()
        model.optimize()
        print('%s blocked (bounds: %s), new growth rate %f' %
              (reaction.id, str(reaction.bounds), model.objective.value))

10FTHF5GLUtl blocked (bounds: (0, 0)), new growth rate 0.000000
10FTHF5GLUtm blocked (bounds: (0, 0)), new growth rate 0.000000
10FTHF6GLUtl blocked (bounds: (0, 0)), new growth rate 0.000000
10FTHF6GLUtm blocked (bounds: (0, 0)), new growth rate 0.000000
10FTHF7GLUtl blocked (bounds: (0, 0)), new growth rate 0.000000
10FTHF7GLUtm blocked (bounds: (0, 0)), new growth rate 0.000000
10FTHFtl blocked (bounds: (0, 0)), new growth rate 0.000000
10FTHFtm blocked (bounds: (0, 0)), new growth rate 0.000000
11DOCRTSLtm blocked (bounds: (0, 0)), new growth rate 0.000000
11DOCRTSLtr blocked (bounds: (0, 0)), new growth rate 0.000000
11DOCRTSTRNtm blocked (bounds: (0, 0)), new growth rate 0.000000
11DOCRTSTRNtr blocked (bounds: (0, 0)), new growth rate 0.000000
12HPETATP blocked (bounds: (0, 0)), new growth rate 0.000000
12HPETUPKt blocked (bounds: (0, 0)), new growth rate 0.000000
13DAMPPOX blocked (bounds: (0, 0)), new growth rate 0.000000
15HPETATP blocked (bounds: (0, 0)), new growth rate 0.00

3HAD203n6m blocked (bounds: (0, 0)), new growth rate 0.000000
3HAD203n6p blocked (bounds: (0, 0)), new growth rate 0.000000
3HAD204n3m blocked (bounds: (0, 0)), new growth rate 0.000000
3HAD204n3p blocked (bounds: (0, 0)), new growth rate 0.000000
3HAD204n6m blocked (bounds: (0, 0)), new growth rate 0.000000
3HAD204n6p blocked (bounds: (0, 0)), new growth rate 0.000000
3HAD205n3m blocked (bounds: (0, 0)), new growth rate 0.000000
3HAD205n3p blocked (bounds: (0, 0)), new growth rate 0.000000
3HAD220p blocked (bounds: (0, 0)), new growth rate 0.000000
3HAD221n9p blocked (bounds: (0, 0)), new growth rate 0.000000
3HAD224n6m blocked (bounds: (0, 0)), new growth rate 0.000000
3HAD224n6p blocked (bounds: (0, 0)), new growth rate 0.000000
3HAD2251n3m blocked (bounds: (0, 0)), new growth rate 0.000000
3HAD2251n3p blocked (bounds: (0, 0)), new growth rate 0.000000
3HAD226n3m blocked (bounds: (0, 0)), new growth rate 0.000000
3HAD226n3p blocked (bounds: (0, 0)), new growth rate 0.000000
3HAD240p

3HDH81n3m blocked (bounds: (0, 0)), new growth rate 0.000000
3HDH81n3p blocked (bounds: (0, 0)), new growth rate 0.000000
3HDH90m blocked (bounds: (0, 0)), new growth rate 0.000000
3HDHDMHPm blocked (bounds: (0, 0)), new growth rate 0.000000
3HDHDMNm blocked (bounds: (0, 0)), new growth rate 0.000000
3HDHMPm blocked (bounds: (0, 0)), new growth rate 0.000000
3HDHPRISTp blocked (bounds: (0, 0)), new growth rate 0.000000
3HDHTMHDp blocked (bounds: (0, 0)), new growth rate 0.000000
3HDHTMTDp blocked (bounds: (0, 0)), new growth rate 0.000000
3HKYNAKGAT blocked (bounds: (0, 0)), new growth rate 0.000000
3HKYNAKGATm blocked (bounds: (0, 0)), new growth rate 0.000000
3HLYTCL blocked (bounds: (0, 0)), new growth rate 0.000000
3HPRODHx blocked (bounds: (0, 0)), new growth rate 0.000000
3HXKYNDCL blocked (bounds: (0, 0)), new growth rate 0.000000
3HXKYNOXDA blocked (bounds: (0, 0)), new growth rate 0.000000
3KLCGAH blocked (bounds: (0, 0)), new growth rate 0.000000
3M2OPLOXRD blocked (bounds: (

ABC_TCDBd blocked (bounds: (0, 0)), new growth rate 0.000000
ABO1g blocked (bounds: (0, 0)), new growth rate 0.000000
ABO2g blocked (bounds: (0, 0)), new growth rate 0.000000
ABO3g blocked (bounds: (0, 0)), new growth rate 0.000000
ABO4g blocked (bounds: (0, 0)), new growth rate 0.000000
ABO5g blocked (bounds: (0, 0)), new growth rate 0.000000
ABO6g blocked (bounds: (0, 0)), new growth rate 0.000000
ABO7g blocked (bounds: (0, 0)), new growth rate 0.000000
ABO8g blocked (bounds: (0, 0)), new growth rate 0.000000
ABO9g blocked (bounds: (0, 0)), new growth rate 0.000000
ABTArm blocked (bounds: (0, 0)), new growth rate 0.000000
ABTD blocked (bounds: (0, 0)), new growth rate 0.000000
ABTti blocked (bounds: (0, 0)), new growth rate 0.000000
ABUTD blocked (bounds: (0, 0)), new growth rate 0.000000
ABUTDm blocked (bounds: (0, 0)), new growth rate 0.000000
ABUTt2r blocked (bounds: (0, 0)), new growth rate 0.000000
ABUTt2rL blocked (bounds: (0, 0)), new growth rate 0.000000
ABUTt4_2_r blocked (b

ACGALtlg blocked (bounds: (0, 0)), new growth rate 0.000000
ACGALtly blocked (bounds: (0, 0)), new growth rate 0.000000
ACGAM2E blocked (bounds: (0, 0)), new growth rate 0.000000
ACGAM6PSi blocked (bounds: (0, 0)), new growth rate 0.000000
ACGAMK blocked (bounds: (0, 0)), new growth rate 0.000000
ACGAMPM blocked (bounds: (0, 0)), new growth rate 0.000000
ACGAMtg blocked (bounds: (0, 0)), new growth rate 0.000000
ACGAMtl blocked (bounds: (0, 0)), new growth rate 0.000000
ACGAMtly blocked (bounds: (0, 0)), new growth rate 0.000000
ACGBGBSIDEtg blocked (bounds: (0, 0)), new growth rate 0.000000
ACGBGBSIDEtl blocked (bounds: (0, 0)), new growth rate 0.000000
ACGLUte blocked (bounds: (0, 0)), new growth rate 0.000000
ACGLUtm blocked (bounds: (0, 0)), new growth rate 0.000000
ACGPID blocked (bounds: (0, 0)), new growth rate 0.000000
ACGSm blocked (bounds: (0, 0)), new growth rate 0.000000
ACHEe blocked (bounds: (0, 0)), new growth rate 0.000000


KeyboardInterrupt: 

In [10]:
import tempfile
from pprint import pprint
from cobra.io import write_sbml_model, validate_sbml_model
with tempfile.NamedTemporaryFile(suffix='.xml') as f_sbml:
    write_sbml_model(model, filename=f_sbml.name)
    report = validate_sbml_model(filename=f_sbml.name)

pprint(report)

TypeError: in method 'Reaction_setReversible', argument 2 of type 'bool'